<a href="https://colab.research.google.com/github/ipsi1999/DeepLearning_LogiGAN/blob/main/764_LOGIGAN_phase1_Baseline%20Adversarial%20Fine-Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 0) Install dependencies
# Run this once in Colab
!pip install -q transformers datasets sentencepiece spacy sklearn
!python -m spacy download en_core_web_sm

# 1) Imports & seeds
import random
import re
import os
import math
from tqdm.auto import tqdm
import numpy as np
import pandas as pd

import spacy
nlp = spacy.load("en_core_web_sm")

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset, load_dataset

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# set seeds for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print("Torch device:", torch.device("cuda" if torch.cuda.is_available() else "cpu"))

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 60.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
Torch device: cpu


In [ ]:
# 2) Load a small corpus (AG News) and build sentence list
# We'll load a small slice for speed and split each example into sentences
raw = load_dataset("ag_news", split="train[:2000]")  # 2000 articles -> many sentences
texts = []
for item in raw:
    full = item["text"]
    # simple sentence splitting via spaCy
    doc = nlp(full)
    for sent in doc.sents:
        s = sent.text.strip()
        # keep only reasonably long sentences to have meaningful structure
        if len(s) > 20 and len(s) < 300:
            texts.append(s)

print(f"Extracted {len(texts)} sentences from AG News slice.")

# quick random sample to inspect
for i in range(5):
    print("-", texts[i])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Extracted 3045 sentences from AG News slice.
- Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
- Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
- Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
- Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.
- Oil prices soar to all-time record, posing 

In [ ]:
# 3) Heuristic detector for logical sentences
#   - causal: because, since, therefore, thus, hence, so that...
#   - conditional: if, unless, only if, provided that...
#   - contrast: although, however, despite, nevertheless, but
#   - quantifier: all, every, none, some, each
#   - inference markers: implies, suggests, indicates, ergo

LOGIC_WORDS = {
    "causal": ["because", "since", "therefore", "thus", "hence", "so that", "as a result", "consequently"],
    "conditional": ["if", "unless", "only if", "provided that", "when ... then", "in case"],
    "contrast": ["although", "however", "despite", "nevertheless", "but", "yet"],
    "quantifier": ["all", "every", "none", "some", "each", "no", "at least", "at most", "only"],
    "inference": ["implies", "suggests", "indicates", "ergo", "therefore"],
}

# Compile a flat set for quick membership checks
flat_words = set()
for k,v in LOGIC_WORDS.items():
    for phrase in v:
        flat_words.add(phrase)

# helper to detect if any of the phrases appears as a word/phrase
def contains_phrase(text, phrase):
    # naive check: phrase may contain spaces, use regex with word boundaries
    pattern = r"\b" + re.escape(phrase) + r"\b"
    return re.search(pattern, text, flags=re.IGNORECASE) is not None

def is_logical_sentence(text):
    text_low = text.lower()
    # check token-level and phrase-level presence
    for phrase in flat_words:
        if contains_phrase(text_low, phrase):
            return True
    # dependency-level heuristics: presence of mark/adverbial clause like 'if' 'because' etc
    doc = nlp(text)
    for token in doc:
        if token.dep_ in ("mark", "advcl") and token.text.lower() in {"if", "because", "although", "since"}:
            return True
    # a fallback: sentences with at least two clauses often have reasoning; count commas + 'that'
    if text.count(",") >= 1 and ("that" in text_low or "so" in text_low):
        return True
    return False

# Tag the sentences
labels = [1 if is_logical_sentence(t) else 0 for t in texts]
df = pd.DataFrame({"text": texts, "label": labels})
print(df.label.value_counts())

# Inspect some positive & negative examples to validate heuristics
print("\nSOME POSITIVE SAMPLES (label=1):")
print(df[df.label==1].sample(5, random_state=SEED)["text"].tolist())

print("\nSOME NEGATIVE SAMPLES (label=0):")
print(df[df.label==0].sample(5, random_state=SEED)["text"].tolist())


label
0    2103
1     942
Name: count, dtype: int64

SOME POSITIVE SAMPLES (label=1):
['The United States lost their first  basketball match at the Olympics since 1988 on Sunday when  Puerto Rico gave them a shock 92-73 trouncing.', 'Interwoven scoops up records management company ECM (enterprise content management) company Interwoven on Monday announced the acquisition of Software Intelligence, a maker of records management technology, for \\$2 million in cash and stock.', 'Holiday Pals Berlusconi, Blair Cement Ties With a bandanna on his head and a grin on his cheeks, Italy #39;s flamboyant Premier Silvio Berlusconi has been playing vacation host to Britain #39;s Tony Blair in Sardinia, an unlikely holiday coupling that highlights an Italian-British alliance ...', 'In an upset as historic as it was inevitable, Tim Duncan, Allen Iverson and the rest of the U.S. basketball team lost 92-73 to Puerto Rico on Sunday, only the third Olympic loss ever for America and its first since adding 

In [ ]:
# 4) Build adversarial/perturbed examples
# We create transformations:
#   A) flip_connective: 'because' -> 'although' or 'therefore' -> 'but' (makes reasoning suspect)
#   B) remove_connective: drop 'because', 'therefore', 'if' -> often destroys valid logic signal
#   C) simple_negation: insert 'not' after first auxiliary verb (is/are/was/will/can), or prepend "It is not the case that "
# Note: these are heuristic and intentionally simple.

AUX_VERBS = ["is", "are", "was", "were", "has", "have", "had", "will", "can", "could", "should", "would", "do", "does", "did", "may", "might"]

def flip_connective(text):
    mapping = {
        "because": "although",
        "therefore": "but",
        "thus": "but",
        "hence": "but",
        "if": "unless",
        "although": "because",  # sometimes flips
        "since": "although",
    }
    text_low = text.lower()
    for src, tgt in mapping.items():
        if contains_phrase(text_low, src):
            # replace only first occurrence (preserve case roughly)
            pattern = re.compile(r"\b" + re.escape(src) + r"\b", flags=re.IGNORECASE)
            return pattern.sub(tgt, text, count=1)
    return text

def remove_connective(text):
    # remove common connectives
    to_remove = ["because", "therefore", "thus", "hence", "if", "although", "since", "so that", "consequently"]
    text2 = text
    for r in to_remove:
        text2 = re.sub(r"\b" + re.escape(r) + r"\b", "", text2, flags=re.IGNORECASE)
    # clean multiple spaces
    text2 = re.sub(r"\s+", " ", text2).strip()
    return text2

def simple_negate(text):
    doc = nlp(text)
    # try to insert 'not' after first auxiliary verb
    for token in doc:
        if token.text.lower() in AUX_VERBS:
            # insert 'not' after this token in the original text string
            # locate token in string by char offsets
            start = token.idx + len(token.text)
            neg_text = text[:start] + " not" + text[start:]
            return neg_text
    # fallback: prepend "It is not the case that "
    return "It is not the case that " + text

# Test transformations
examples = df[df.label==1].sample(6, random_state=SEED)["text"].tolist()
for ex in examples:
    print("\nORIG:", ex)
    print("FLIP:", flip_connective(ex))
    print("REMOVE:", remove_connective(ex))
    print("NEGATE:", simple_negate(ex))



ORIG: The United States lost their first  basketball match at the Olympics since 1988 on Sunday when  Puerto Rico gave them a shock 92-73 trouncing.
FLIP: The United States lost their first  basketball match at the Olympics although 1988 on Sunday when  Puerto Rico gave them a shock 92-73 trouncing.
REMOVE: The United States lost their first basketball match at the Olympics 1988 on Sunday when Puerto Rico gave them a shock 92-73 trouncing.
NEGATE: It is not the case that The United States lost their first  basketball match at the Olympics since 1988 on Sunday when  Puerto Rico gave them a shock 92-73 trouncing.

ORIG: Interwoven scoops up records management company ECM (enterprise content management) company Interwoven on Monday announced the acquisition of Software Intelligence, a maker of records management technology, for \$2 million in cash and stock.
FLIP: Interwoven scoops up records management company ECM (enterprise content management) company Interwoven on Monday announced th

In [ ]:
# 5) Create balanced dataset for classification
# We'll create: original logical positives (label=1), random negatives (label=0)
pos_texts = df[df.label==1]["text"].tolist()
neg_texts = df[df.label==0]["text"].tolist()

# sample sizes
N_pos = min(1200, len(pos_texts))   # cap for speed
N_neg = N_pos

# sample
pos_sample = random.sample(pos_texts, N_pos) if len(pos_texts) > N_pos else pos_texts
neg_sample = random.sample(neg_texts, N_neg) if len(neg_texts) > N_neg else neg_texts[:N_neg]

print(f"\nUsing {len(pos_sample)} positive and {len(neg_sample)} negative samples for baseline.")

# Prepare baseline dataset (text, label)
baseline_texts = pos_sample + neg_sample
baseline_labels = [1]*len(pos_sample) + [0]*len(neg_sample)

# Shuffle
combined = list(zip(baseline_texts, baseline_labels))
random.shuffle(combined)
baseline_texts, baseline_labels = zip(*combined)

# Build HuggingFace Dataset
ds_baseline = Dataset.from_dict({"text": list(baseline_texts), "label": list(baseline_labels)})
ds_baseline = ds_baseline.train_test_split(test_size=0.2, seed=SEED)

print(ds_baseline)



Using 942 positive and 942 negative samples for baseline.
DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1507
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 377
    })
})


In [ ]:
# 6) Tokenizer & model (DistilBERT)
MODEL_NAME = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def preprocess(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=128)

tokenized_train = ds_baseline["train"].map(preprocess, batched=True)
tokenized_eval  = ds_baseline["test"].map(preprocess, batched=True)
tokenized_train = tokenized_train.remove_columns(["text"])
tokenized_eval = tokenized_eval.remove_columns(["text"])

# set format for Trainer
tokenized_train.set_format("torch")
tokenized_eval.set_format("torch")

# 7) Training function wrapper (so we can run baseline then adversarial)
def train_model(train_dataset, eval_dataset, output_dir, num_train_epochs=2):
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=16 if torch.cuda.is_available() else 8,
        per_device_eval_batch_size=32 if torch.cuda.is_available() else 16,
        eval_strategy="epoch",
        save_strategy="no",
        learning_rate=2e-5,
        weight_decay=0.01,
        logging_steps=50,
        seed=SEED,
        report_to="none",
    )
    def compute_metrics(pred):
        labels = pred.label_ids
        preds = np.argmax(pred.predictions, axis=1)
        return {
            "accuracy": float(accuracy_score(labels, preds)),
            "f1": float(f1_score(labels, preds)),
            "precision": float(precision_score(labels, preds)),
            "recall": float(recall_score(labels, preds))
        }

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )
    trainer.train()
    metrics = trainer.evaluate()
    return trainer, metrics

Map:   0%|          | 0/1507 [00:00<?, ? examples/s]

Map:   0%|          | 0/377 [00:00<?, ? examples/s]

In [ ]:
# 8) Baseline training
print("\n=== TRAINING BASELINE MODEL (Logical vs Non-Logical) ===")
baseline_trainer, baseline_metrics = train_model(tokenized_train, tokenized_eval, output_dir="./baseline_model", num_train_epochs=2)
print("Baseline eval metrics:", baseline_metrics)



=== TRAINING BASELINE MODEL (Logical vs Non-Logical) ===


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-3770604330.py:43: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.608900,0.538089,0.742706,0.772834,0.673469,0.906593
2,0.495400,0.463024,0.790451,0.787062,0.772487,0.802198


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Baseline eval metrics: {'eval_loss': 0.4630238711833954, 'eval_accuracy': 0.7904509283819628, 'eval_f1': 0.7870619946091644, 'eval_precision': 0.7724867724867724, 'eval_recall': 0.8021978021978022, 'eval_runtime': 90.8017, 'eval_samples_per_second': 4.152, 'eval_steps_per_second': 0.264, 'epoch': 2.0}


In [ ]:
# 9) Create adversarial dataset
# We'll take positive logical sentences and create adversarial variants (flip, remove, negate)
adv_texts = []
adv_labels = []

# Use a subset to generate adversarials to avoid explosion
N_adv_gen = min(800, len(pos_sample))
pos_for_adv = random.sample(pos_sample, N_adv_gen)

for t in pos_for_adv:
    # original positive kept as positive label 1
    adv_texts.append(t); adv_labels.append(1)
    # flipped connective -> likely breaks the logical flow --> treat as negative (0)
    flipped = flip_connective(t)
    if flipped != t:
        adv_texts.append(flipped); adv_labels.append(0)
    # removed connective -> often removes the explicit inference marker -> negative
    removed = remove_connective(t)
    if removed != t and len(removed) > 15:
        adv_texts.append(removed); adv_labels.append(0)
    # negation -> negative
    neg = simple_negate(t)
    if neg != t:
        adv_texts.append(neg); adv_labels.append(0)

# For extra negatives, add random negative samples
extra_negs = random.sample(neg_texts, min(800, len(neg_texts)))
for t in extra_negs:
    adv_texts.append(t); adv_labels.append(0)

# Combine adversarial data with some original neutral data to balance
print(f"Generated {len(adv_texts)} adversarial / augmented examples.")

# Build dataset and split
ds_adv = Dataset.from_dict({"text": adv_texts, "label": adv_labels})
ds_adv = ds_adv.train_test_split(test_size=0.2, seed=SEED)

# Tokenize
tokenized_adv_train = ds_adv["train"].map(preprocess, batched=True)
tokenized_adv_eval  = ds_adv["test"].map(preprocess, batched=True)
tokenized_adv_train = tokenized_adv_train.remove_columns(["text"])
tokenized_adv_eval = tokenized_adv_eval.remove_columns(["text"])
tokenized_adv_train.set_format("torch")
tokenized_adv_eval.set_format("torch")


Generated 2732 adversarial / augmented examples.


Map:   0%|          | 0/2185 [00:00<?, ? examples/s]

Map:   0%|          | 0/547 [00:00<?, ? examples/s]

In [ ]:
# 10) Adversarial fine-tuning:
# We will take the baseline model weights and continue training on augmented adversarial set.
print("\n=== ADVERSARIAL FINE-TUNING ===")
# save baseline model then reload to ensure training starts from baseline weights
baseline_trainer.save_model("./baseline_saved")
adv_model = AutoModelForSequenceClassification.from_pretrained("./baseline_saved", num_labels=2)

training_args_adv = TrainingArguments(
    output_dir="./adv_model",
    num_train_epochs=2,
    per_device_train_batch_size=16 if torch.cuda.is_available() else 8,
    per_device_eval_batch_size=32 if torch.cuda.is_available() else 16,
    eval_strategy="epoch",
    save_strategy="no",
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=50,
    seed=SEED,
    report_to="none",
)

def compute_metrics_adv(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    return {
        "accuracy": float(accuracy_score(labels, preds)),
        "f1": float(f1_score(labels, preds)),
        "precision": float(precision_score(labels, preds)),
        "recall": float(recall_score(labels, preds))
    }

trainer_adv = Trainer(
    model=adv_model,
    args=training_args_adv,
    train_dataset=tokenized_adv_train,
    eval_dataset=tokenized_adv_eval,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics_adv
)

trainer_adv.train()
adv_metrics = trainer_adv.evaluate()
print("Adversarial-finetune eval metrics:", adv_metrics)



=== ADVERSARIAL FINE-TUNING ===


/tmp/ipython-input-3686415669.py:32: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_adv = Trainer(
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.475500,0.480928,0.744059,0.523810,0.534722,0.513333
2,0.371200,0.481532,0.760512,0.578778,0.559006,0.600000


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Adversarial-finetune eval metrics: {'eval_loss': 0.4815317392349243, 'eval_accuracy': 0.7605118829981719, 'eval_f1': 0.5787781350482315, 'eval_precision': 0.5590062111801242, 'eval_recall': 0.6, 'eval_runtime': 132.6709, 'eval_samples_per_second': 4.123, 'eval_steps_per_second': 0.264, 'epoch': 2.0}


In [ ]:

# 11) Evaluate both models on a held-out test set (we'll use baseline's test for consistent comparison)
# Get predictions from baseline model on baseline test
def get_preds(trainer, tokenized_dataset):
    preds_output = trainer.predict(tokenized_dataset)
    preds = np.argmax(preds_output.predictions, axis=1)
    labels = preds_output.label_ids
    return preds, labels, preds_output.metrics

baseline_preds, baseline_labels, baseline_eval_metrics = get_preds(baseline_trainer, tokenized_eval)
adv_preds, adv_labels, adv_eval_metrics = get_preds(trainer_adv, tokenized_eval)  # evaluate adv model on the same baseline eval set

print("\nEVALUATION ON BASELINE EVAL SET (same held-out split):")
print("Baseline model metrics (reported earlier):", baseline_eval_metrics)
print("Adv-finetuned model evaluated on same set (to test generalization):", adv_eval_metrics)

# Compute comparative metrics (accuracy/f1) directly for the eval set
print("\nDirect comparison on baseline eval set:")
print("Baseline accuracy:", accuracy_score(baseline_labels, baseline_preds), "F1:", f1_score(baseline_labels, baseline_preds))
print("Adv model accuracy:", accuracy_score(baseline_labels, adv_preds), "F1:", f1_score(baseline_labels, adv_preds))


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)



EVALUATION ON BASELINE EVAL SET (same held-out split):
Baseline model metrics (reported earlier): {'test_loss': 0.4630238711833954, 'test_accuracy': 0.7904509283819628, 'test_f1': 0.7870619946091644, 'test_precision': 0.7724867724867724, 'test_recall': 0.8021978021978022, 'test_runtime': 90.1419, 'test_samples_per_second': 4.182, 'test_steps_per_second': 0.266}
Adv-finetuned model evaluated on same set (to test generalization): {'test_loss': 0.5017519593238831, 'test_accuracy': 0.8037135278514589, 'test_f1': 0.7771084337349398, 'test_precision': 0.86, 'test_recall': 0.7087912087912088, 'test_runtime': 89.32, 'test_samples_per_second': 4.221, 'test_steps_per_second': 0.269}

Direct comparison on baseline eval set:
Baseline accuracy: 0.7904509283819628 F1: 0.7870619946091644
Adv model accuracy: 0.8037135278514589 F1: 0.7771084337349398


In [ ]:
# 12) Qualitative inspection: show sample predictions side-by-side
def show_samples(texts, true_labels, base_preds, adv_preds, n=12):
    df = pd.DataFrame({
        "text": texts,
        "true": true_labels,
        "base_pred": base_preds,
        "adv_pred": adv_preds
    })
    display(df.sample(n))

# Reconstruct texts from tokenized_eval (we removed the 'text' column earlier)
# We still have original baseline split in ds_baseline["test"] -> use that
test_texts = ds_baseline["test"]["text"]
show_samples(test_texts, baseline_labels, baseline_preds, adv_preds, n=10)

,text,true,base_pred,adv_pred
286,A lot of developers shorten this to:\\ stat...,1,1,1
258,New Method May Predict Quakes Weeks Ahead (AP)...,1,1,1
262,"They reported this to the Windows\developers, ...",1,1,1
145,CA buys PestPatrol antispyware developer WASHI...,1,1,1
55,Intel to Delay Product for High-Definition TVs...,0,0,0
93,This requires long range autonomous strike wea...,0,0,0
339,Bell set for Test debut Ian Bell is almost cer...,0,0,0
82,"Intel delays first TV chip In another setback,...",1,1,0
364,Unobstructed Views: Olympic-sized fools For al...,1,1,1
148,"Singh Leads, but Leonard Is Following Avoiding...",1,1,1


In [ ]:
# 13) Save final adversarial model for submission
trainer_adv.save_model("./final_adv_model")
print("Saved final adversarial model to ./final_adv_model")


Saved final adversarial model to ./final_adv_model


In [ ]:
# 14) Quick conclusions to print
print("\nCONCLUSIONS / NEXT STEPS:")
print("- Baseline vs adversarial-fine-tuned metrics shown above.")
print("- Qualitatively inspect misclassifications to refine adversarial generation rules.")
print("- To emulate LogiGAN more, you could build masked-generation tasks (harder) or use multi-step chain examples.")


CONCLUSIONS / NEXT STEPS:
- Baseline vs adversarial-fine-tuned metrics shown above.
- Qualitatively inspect misclassifications to refine adversarial generation rules.
- To emulate LogiGAN more, you could build masked-generation tasks (harder) or use multi-step chain examples.
